In [1]:
import numpy as np
import warnings

<h2> Functions </h2>

In [2]:
def write_revision(file_name, n_addr, wordSize):    
    revision_string = f"""
    -------------------------------------------------------------------
    -- File   : {file_name}
    -------------------------------------------------------------------
    -- Description : Synchronous {n_addr} x {wordSize} ROM.
    -------------------------------------------------------------------
    -- Revision:
    --     Date        Rev     Author                      Description
    --     12/07/2023  1.0     Henrique Lefundes da Silva  Creation.
    -------------------------------------------------------------------"""

    return revision_string

def write_entity(entity_name, n_bits_addr, wordSize):
    entity_string = f"""
    library ieee;
    use ieee.std_logic_1164.all;
    use ieee.numeric_std.all;

    entity {entity_name} is
    port (
            address   : in  std_logic_vector({n_bits_addr-1} downto 0);
            clk       : in  std_logic;
            out_rom   : out std_logic_vector({wordSize-1} downto 0)
        );
    end entity {entity_name};
    """

    return entity_string

def write_architecture_head(entity_name, n_addr, wordSize):  
    architecture_head_string = f"""
    -- Initial data explicitly declared
    architecture {entity_name}_arch of {entity_name} is
      type memory_struct is array(0 to {n_addr-1}) of std_logic_vector({wordSize-1} downto 0);
    """
    return architecture_head_string

def write_memory(rom_values, wordSize):
  memory_string = """
    signal memory : memory_struct := ({});
  """

  # The format command casts the values from integers to binaries.
  # The binary number must have wordSize bits, so zeros are padded to the left.
  # The join function will skip lines, put the commas and tab the code correctly.

  memory_string = memory_string.format(',\n \t \t \t \t'.join([f'"{format(x, f"0{wordSize}b")}"' for x in rom_values]), format(0, f"0{wordSize}b"))

  return memory_string
  
def write_architecture_body(entity_name):
  architecture_body_string = f"""
    begin
    -- Reads ROM position at every rising edge of 'clk'.
    process (clk)
    begin
    if (clk'event and clk = '1') then
        out_rom <= memory(to_integer(unsigned(address)));
      end if;
    end process;

  end architecture {entity_name}_arch;  
  """
  
  return architecture_body_string

In [3]:
def write_rom(file_name, entity_name, n_addr, wordSize, rom_values):
    '''
    This generates a ROM implemented in VHDL.

    Parameters
    ----------------
    file_name: string
        The name of the VHDL file that will be generated.
    
    entity_name: string
        ROM's entity name.

    n_addr: int
        The number of address that the ROM will have. 
    
    wordSize: int
        The size of the word that the ROM will store.
    
    rom_values: np.array
        The values that will be written into the ROM.
    '''

    # Number of bits-1 needed to reach the number of addresses
    
    n_bits_addr = np.int8(np.ceil(np.log2(n_addr)))

    # Verify if n_addr is divisible by 2^n. If not, append zeros to the values until you get a multiple.
    if not np.log2(n_addr).is_integer():
        while rom_values.shape[0] < 2**n_bits_addr:
            rom_values = np.append(rom_values, 0)
            n_addr = 2**n_bits_addr
        warnings.warn("The number of addresses it not an exponent of 2. You may have addresses with value output to zero.")
    
    # Verify if the lengths match, if not append zeros to the end
    if rom_values.shape[0] < n_addr:
        while rom_values.shape[0] < n_addr:
            rom_values = np.append(rom_values, 0)
        warnings.warn("You have more addresses than values. The list will be appended with zeros to match addresses' length.")

    revision_string = write_revision(file_name, n_addr, wordSize)
    entity_string = write_entity(entity_name, n_bits_addr, wordSize)
    architecture_head_string = write_architecture_head(entity_name, n_addr, wordSize)
    memory_string = write_memory(rom_values, wordSize)
    architecture_body_string = write_architecture_body(entity_name)

    # Concatenate the strings
    VHDL = revision_string + entity_string + architecture_head_string + memory_string + architecture_body_string

    # Write the file
    fout = open(f'{file_name}.vhd', 'w')

    # Write the string into the file
    fout.write(VHDL)

    # Close the file
    fout.close()

    # Return the complete string
    return VHDL

<h2> Example </h2>

In [4]:
# Arquivo Exemplo

# Final VHDL file name
file_name = "stimulus_list_example" 

# ROM Entity name
entity_name = file_name

# ROM Entity name
n_addr = 100

# Word size of the memory
wordSize = 24

# The values that will be written
rom_values = np.random.randint(0, 2**wordSize, n_addr-10)

# Write the ROM
result = write_rom(file_name, entity_name, n_addr, wordSize, rom_values)

# Print result
print(result)


    -------------------------------------------------------------------
    -- File   : stimulus_list_example
    -------------------------------------------------------------------
    -- Description : Synchronous 128 x 24 ROM.
    -------------------------------------------------------------------
    -- Revision:
    --     Date        Rev     Author                      Description
    --     12/07/2023  1.0     Henrique Lefundes da Silva  Creation.
    -------------------------------------------------------------------
    library ieee;
    use ieee.std_logic_1164.all;
    use ieee.numeric_std.all;

    entity stimulus_list_example is
    port (
            address   : in  std_logic_vector(6 downto 0);
            clk       : in  std_logic;
            out_rom   : out std_logic_vector(23 downto 0)
        );
    end entity stimulus_list_example;
    
    -- Initial data explicitly declared
    architecture stimulus_list_example_arch of stimulus_list_example is
      type memory_s

C:\Users\R2D2\AppData\Local\Temp\ipykernel_1660\2775280496.py:32: UserWarning: The number of addresses it not an exponent of 2. You may have addresses with value output to zero.
  warnings.warn("The number of addresses it not an exponent of 2. You may have addresses with value output to zero.")


<h2> Write ROM for the inputs </h2>

In [5]:
import pandas as pd

# Reads the dataset
# The DatasetT3 was used because it's the same dataset used for HW/SW comparison.
data = pd.read_csv("DatasetT3.csv", names = ["Distance", "Speed", "Deceleration", "Decision"])

# Get the list of indexes for corner cases in DatasetT3
corner_cases_indexes = pd.read_csv("DatasetT3Corner.txt", header=None)
display(data)

# The number of random values (the number of address in the ROM)
n_addr = 94

# Word size of the memory
wordSize = 24

# Generate random indexes
# The interval was picked using probability distribution.
# Where we will start our search
start = 1150
# Number of distributions we will pick
n_distribution = 4
# Number of reserved addresses for corner case situations
n_corner_cases = 15
# The indexes array
indexes = []
# The number of rand values per distribution that we must generate
n_rand = np.int8((n_addr-n_corner_cases)/n_distribution)

#These intervals has prob values between 0 and 1, then we have higher chances of picking good values instead of zeros or ones only.
for i in range(n_distribution):
    # Generate n_rand random integers
    rand = np.random.randint(1150, 1150+70, n_rand)
    # Write them in our array
    indexes.extend(rand)

# Get corner cases from the list (randomly)
corner_cases_indexes_reduced = corner_cases_indexes.sample(n_corner_cases).to_numpy()

# Put them into our array
indexes.extend(corner_cases_indexes_reduced)

# Copies the last index if the shape does not match
while len(indexes) < n_addr:
    indexes.append(indexes[-1])

# Cast to numpy array
indexes = np.array(indexes)

# Cast all values to integers
indexes = np.uint16(indexes)

# Cast dataset to numpy
data_np = data.to_numpy()

# Get the data picked randomly (debug only)
data_sorted = [data.iloc[index] for index in indexes]
display(data_sorted)

# Transform the values to integer
data_np = np.int32((data_np*(2**12)))

# Creates the lists
distance_list = np.zeros(n_addr)
speed_list = np.zeros(n_addr)
deceleration_list = np.zeros(n_addr)
decision_list = np.zeros(n_addr)

# Put the random values into the list
for i in range(n_addr):
    distance_list[i] = data_np[indexes[i], 0] 
    speed_list[i] = data_np[indexes[i], 1] 
    deceleration_list[i] = data_np[indexes[i], 2] 
    decision_list[i] = data_np[indexes[i], 3] 

# Show the results (debug only)
print(distance_list)
print(speed_list)
print(deceleration_list)

############## DISTANCE ROM ##############

# Final VHDL file name
file_name = "stimulus_list_distance" 

# ROM Entity name
entity_name = file_name

# The values on the list must be integers
distance_list = np.uint32(distance_list)

# Write the ROM
result = write_rom(file_name, entity_name, n_addr, wordSize, distance_list)


############## SPEED ROM ##############

# Final VHDL file name
file_name = "stimulus_list_speed" 

# ROM Entity name
entity_name = file_name

# The values on the list must be integers
speed_list = np.uint32(speed_list)

# Write the ROM
result = write_rom(file_name, entity_name, n_addr, wordSize, speed_list)


############## DECELERATION ROM ##############

# Final VHDL file name
file_name = "stimulus_list_deceleration" 

# ROM Entity name
entity_name = file_name

# The values on the list must be integers
deceleration_list = np.uint32(deceleration_list)

# Write the ROM
result = write_rom(file_name, entity_name, n_addr, wordSize, deceleration_list)

,Distance,Speed,Deceleration,Decision
0,0.504450,0.000000,1.3950,1.0
1,0.000000,0.000000,1.1625,1.0
2,0.000000,0.000000,0.9300,1.0
3,1.158221,0.000000,0.7800,1.0
4,0.156073,0.000000,0.6500,1.0
...,...,...,...,...
57280,1998.864111,26.913394,1.3950,0.0
57281,1998.304338,27.914276,1.1625,0.0
57282,1999.635227,27.073422,0.9300,0.0
57283,1999.776292,27.574239,0.7800,0.0


C:\Users\R2D2\AppData\Local\Temp\ipykernel_1660\3015639929.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  indexes = np.array(indexes)


[Distance        39.814389
 Speed            5.027790
 Deceleration     0.930000
 Decision         0.000000
 Name: 1192, dtype: float64,
 Distance        40.605776
 Speed            6.989825
 Deceleration     1.162500
 Decision         1.000000
 Name: 1211, dtype: float64,
 Distance        40.291297
 Speed            5.379688
 Deceleration     0.930000
 Decision         0.000000
 Name: 1197, dtype: float64,
 Distance        42.489556
 Speed            6.481899
 Deceleration     1.162500
 Decision         1.000000
 Name: 1206, dtype: float64,
 Distance        41.492368
 Speed            1.016011
 Deceleration     1.395000
 Decision         0.000000
 Name: 1150, dtype: float64,
 Distance        40.168106
 Speed            6.907781
 Deceleration     0.650000
 Decision         1.000000
 Name: 1214, dtype: float64,
 Distance        36.760000
 Speed            1.001986
 Deceleration     0.930000
 Decision         0.000000
 Name: 1152, dtype: float64,
 Distance        43.202012
 Speed        

[ 163079.  166321.  165033.  174037.  169952.  164528.  150568.  176955.
  166321.  164528.  162984.  169705.  162870.  164665.  165172.  163705.
  154178.  163970.  162376.  174003.  157223.  169257.  159854.  164528.
  167057.  176955.  176214.  169952.  176955.  150568.  169952.  165821.
  165162.  176214.  168847.  170253.  161411.  163315.  177111.  166052.
  164528.  163970.  160380.  161694.  162984.  160308.  169349.  166519.
  152850.  177111.  167057.  170312.  174039.  161702.  161702.  169257.
  170312.  166052.  158496.  165033.  163315.  162376.  160380.  177111.
  163079.  159321.  174039.  167758.  170874.  165033.  164903.  162851.
  163315.  169349.  169616.  153988. 1426280. 1482827. 1760060. 1815511.
 2003176. 1887895. 1710129. 1469751. 1142451.  561222.  519208. 1364706.
 1511216. 2215883. 1236724. 1236724. 1236724. 1236724.]
[ 20593.  28630.  22035.  26549.   4161.  28294.   4104.  19235.  28630.
  28294.   9840.  17186.  32282.  10368.  20146.  14045.  11834.   3

C:\Users\R2D2\AppData\Local\Temp\ipykernel_1660\2775280496.py:32: UserWarning: The number of addresses it not an exponent of 2. You may have addresses with value output to zero.
  warnings.warn("The number of addresses it not an exponent of 2. You may have addresses with value output to zero.")
